# 2. Modélisation avec la Régression Logistique

## 2.1. Introduction

Ce notebook présente une approche complète de classification des sentiments utilisant la **Régression Logistique**. C'est un excellent modèle de base pour les tâches de classification de texte.

**Étapes Principales :**
- Chargement des données préparées.
- Encodage des labels.
- Vectorisation des textes avec TF-IDF.
- Optimisation des hyperparamètres avec GridSearchCV.
- Entraînement et évaluation du modèle final.
- Sauvegarde du modèle et des transformations pour une utilisation future.

## 2.2. Importation des bibliothèques

In [ ]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## 2.3. Chargement et Préparation des Données

In [ ]:
# Charger le jeu de données équilibré créé dans le notebook précédent
# Le chemin est relatif au dossier 'notebooks/'
df = pd.read_csv('../data/balanced_subset.csv', encoding='utf-8')

# Pour accélérer l'expérimentation, nous prenons un échantillon de 400 000 lignes
df_subset = df.sample(n=400000, random_state=42)

# Vérification des données chargées
print("Aperçu du jeu de données :")
print(df_subset.head())

# Vérification de la taille de l'échantillon
print(f"\nTaille de l'échantillon : {df_subset.shape}")

# Vérifier l'équilibre des classes dans l'échantillon
print("\nDistribution des classes dans l'échantillon :")
print(df_subset['rating'].value_counts())

## 2.4. Encodage des Labels

Les modèles de machine learning nécessitent des entrées numériques. Nous convertissons donc les labels textuels ('Positive', 'Neutral', 'Negative') en nombres (par exemple, 2, 1, 0).

In [ ]:
# Initialisation de l'encodeur
label_encoder = LabelEncoder()

# Transformation des labels
df_subset['rating_encoded'] = label_encoder.fit_transform(df_subset['rating'])

# Vérification des classes encodées (0: Negative, 1: Neutral, 2: Positive)
print(f"Classes originales : {label_encoder.classes_}")
print(f"Classes encodées : {label_encoder.transform(label_encoder.classes_)}")

## 2.5. Vectorisation des Textes avec TF-IDF

Nous transformons le texte brut en une matrice de caractéristiques numériques en utilisant la technique TF-IDF (Term Frequency-Inverse Document Frequency).

In [ ]:
# Initialisation du TfidfVectorizer
# max_features=10000 limite le vocabulaire aux 10 000 mots les plus fréquents pour réduire la dimensionnalité
tfidf_vectorizer = TfidfVectorizer(max_features=10000)

# Vectorisation des textes
X_tfidf = tfidf_vectorizer.fit_transform(df_subset['cleaned_text'])

# Séparation des features (X) et des labels (y)
X = X_tfidf
y = df_subset['rating_encoded']

# Vérification de la forme des données vectorisées
print(f"Forme de la matrice de features (X) : {X.shape}")

## 2.6. Division en Ensembles d'Entraînement et de Test

In [ ]:
# Division des données en 80% pour l'entraînement et 20% pour le test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Vérification des tailles des ensembles
print(f"Taille de l'ensemble d'entraînement : {X_train.shape}")
print(f"Taille de l'ensemble de test : {X_test.shape}")

## 2.7. Optimisation des Hyperparamètres avec GridSearchCV

Nous utilisons une recherche par grille (GridSearch) pour trouver la meilleure combinaison d'hyperparamètres pour notre modèle de Régression Logistique.

In [ ]:
# Définition de la grille de paramètres à tester
param_grid = [
    {
        'penalty': ['l2'],
        'C': [0.1, 1, 10],
        'solver': ['lbfgs', 'sag']
    },
    {
        'penalty': ['l1', 'l2'],
        'C': [0.1, 1, 10],
        'solver': ['saga'],
    }
]

# Initialisation de GridSearchCV avec validation croisée (5-fold)
grid_search = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), param_grid, cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Entraînement de GridSearchCV
grid_search.fit(X_train, y_train)

# Affichage des meilleurs paramètres trouvés
print(f"Meilleurs hyperparamètres : {grid_search.best_params_}")

# Meilleur modèle trouvé par la recherche
best_logreg = grid_search.best_estimator_

## 2.8. Évaluation du Modèle Final

In [ ]:
# Prédictions sur l'ensemble de test avec le meilleur modèle
y_pred = best_logreg.predict(X_test)

# Calcul de la précision
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du meilleur modèle sur l'ensemble de test : {accuracy:.4f}")

# Rapport de classification détaillé
print("\nRapport de classification :")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Matrice de confusion
print("\nMatrice de confusion :")
print(confusion_matrix(y_test, y_pred))

## 2.9. Sauvegarde du Modèle et des Transformations

Nous sauvegardons le modèle entraîné, le vectoriseur TF-IDF et l'encodeur de labels pour pouvoir les réutiliser dans notre application Streamlit sans avoir à tout ré-entraîner.

In [ ]:
# Sauvegarder le modèle, le vectoriseur et l'encodeur dans le dossier approprié
joblib.dump(best_logreg, '../models/logistic_regression/logreg_model.pkl')
joblib.dump(tfidf_vectorizer, '../models/logistic_regression/tfidf_vectorizer.pkl')
joblib.dump(label_encoder, '../models/logistic_regression/label_encoder.pkl')

print("Modèle, vectoriseur et encodeur sauvegardés dans '../models/logistic_regression/'")

## 2.10. Test de Chargement et de Prédiction

Vérifions que les objets sauvegardés peuvent être chargés et utilisés correctement pour faire une prédiction.

In [ ]:
# Charger le modèle, le vectoriseur et l'encodeur depuis les nouveaux chemins
loaded_model = joblib.load('../models/logistic_regression/logreg_model.pkl')
loaded_vectorizer = joblib.load('../models/logistic_regression/tfidf_vectorizer.pkl')
loaded_label_encoder = joblib.load('../models/logistic_regression/label_encoder.pkl')

# Exemple de texte pour la prédiction
example_text = ["The food was amazing and the service was excellent!"]

# Vectorisation du texte
example_tfidf = loaded_vectorizer.transform(example_text)

# Prédiction
predicted_rating_encoded = loaded_model.predict(example_tfidf)

# Décodage de la prédiction pour obtenir le label textuel
predicted_rating_label = loaded_label_encoder.inverse_transform(predicted_rating_encoded)

print(f"Texte : '{example_text[0]}' -> Prédiction : {predicted_rating_label[0]}")